In [0]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras import backend as K
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GaussianNoise
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 

In [0]:
IMAGE_SIZE = 32
NUM_CLASSES = 10
NUM_EXAMPLES = 60000
SPLIT = 0.10
NUM_EXAMPLES_TRAIN = int(NUM_EXAMPLES * (1 - SPLIT))
NUM_EXAMPLES_TEST = int(NUM_EXAMPLES * SPLIT)

In [0]:
def supervised_model(input_shape=(32, 32, 3), stddev=0.15, dropout=0.5, alpha=0.1, adversarial=False, NUM_CLASSES=10):
	model = Sequential()
	model.add(GaussianNoise(stddev=stddev, input_shape=input_shape))

	model.add(Conv2D(128, kernel_size=(3, 3), padding="same"))
	model.add(LeakyReLU(alpha=alpha))
	model.add(Conv2D(128, kernel_size=(3, 3), padding="same"))
	model.add(LeakyReLU(alpha=alpha))
	model.add(Conv2D(128, kernel_size=(3, 3), padding="same"))
	model.add(LeakyReLU(alpha=alpha))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(dropout))

	model.add(Conv2D(256, kernel_size=(3, 3), padding="same"))
	model.add(LeakyReLU(alpha=alpha))
	model.add(Conv2D(256, kernel_size=(3, 3), padding="same"))
	model.add(LeakyReLU(alpha=alpha))
	model.add(Conv2D(256, kernel_size=(3, 3), padding="same"))
	model.add(LeakyReLU(alpha=alpha))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	model.add(Conv2D(512, kernel_size=(3, 3), padding="valid"))
	model.add(LeakyReLU(alpha=alpha))
	model.add(Conv2D(256, kernel_size=(1, 1)))
	model.add(LeakyReLU(alpha=alpha))
	model.add(Conv2D(128, kernel_size=(1, 1)))
	model.add(LeakyReLU(alpha=alpha))
	model.add(GlobalAveragePooling2D())

	if adversarial:
		#TODO update adversarial droput layer 
		pass
	else: 
		model.add(Dropout(dropout))

	model.add(Dense(NUM_CLASSES, activation='softmax'))

	model.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=keras.optimizers.Adam(),
	              metrics=['accuracy'])
	return model

In [8]:
#@title Dataset Shape
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

170500096/170498071 [==============================] - 39s 0us/step


	# x_train.shape = (50000, 32, 32, 3)
	# y_train.shape = (50000, 1)
	# x_test.shape = (10000, 32, 32, 3)
	# y_test.shape = (10000, 1)
 

In [11]:
model = supervised_model()
model.fit(x_train, y_train, batch_size=64, callbacks=None, epochs=3, verbose=1, validation_data=(x_test, y_test), shuffle=True)
 

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
44224/50000 [=========================>....] - ETA: 12s - loss: 2.2739 - acc: 0.1826

50000/50000 [==============================] - 112s 2ms/step - loss: 2.2285 - acc: 0.1959 - val_loss: 1.8692 - val_acc: 0.2978
Epoch 2/3
26624/50000 [==============>...............] - ETA: 49s - loss: 1.7993 - acc: 0.3570

50000/50000 [==============================] - 111s 2ms/step - loss: 1.6804 - acc: 0.4007 - val_loss: 2.1603 - val_acc: 0.3536
Epoch 3/3
19904/50000 [==========>...................] - ETA: 1:03 - loss: 1.3943 - acc: 0.5077

50000/50000 [==============================] - 111s 2ms/step - loss: 1.2906 - acc: 0.5490 - val_loss: 1.2007 - val_acc: 0.5666
